# Dog Emotion Classifier
This notebook trains a convolutional model on dog emotion images stored in the root `images/` folder.


In [1]:
from pathlib import Path
import json
import random
import time

import torch
from torch import nn
from torch.utils.data import DataLoader, Subset
from torchvision import datasets

from dog_emotion_model import build_transforms, create_model

DATASET_DIR = Path.cwd() / "images"
MODEL_DIR = Path.cwd() / "models"
MODEL_DIR.mkdir(exist_ok=True)
MODEL_PATH = MODEL_DIR / "dog_emotion_cnn.pth"
METADATA_PATH = MODEL_DIR / "metadata.json"

BATCH_SIZE = 32
VAL_SPLIT = 0.2
IMAGE_SIZE = (224, 224)
LEARNING_RATE = 1e-3
EPOCHS = 12
RNG_SEED = 42
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

torch.manual_seed(RNG_SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(RNG_SEED)

assert DATASET_DIR.exists(), f"Dataset folder not found: {DATASET_DIR}"
print(f"Using {DATASET_DIR} with {len(list(DATASET_DIR.rglob('*.*')))} files")
print(f"Training on device: {DEVICE}")


Using h:\DogEmotionn\images with 15921 files
Training on device: cuda


In [2]:
def _build_subsets(dataset_root: Path):
    train_dataset = datasets.ImageFolder(dataset_root, transform=build_transforms(IMAGE_SIZE, augment=True))
    val_dataset = datasets.ImageFolder(dataset_root, transform=build_transforms(IMAGE_SIZE, augment=False))
    total_items = len(train_dataset)
    val_length = max(1, int(total_items * VAL_SPLIT)) if total_items > 1 else 1
    generator = torch.Generator().manual_seed(RNG_SEED)
    shuffled = torch.randperm(total_items, generator=generator)
    val_indices = shuffled[:val_length].tolist()
    train_indices = shuffled[val_length:].tolist()
    if not train_indices:
        train_indices = val_indices
        val_indices = val_indices[:1]
    return Subset(train_dataset, train_indices), Subset(val_dataset, val_indices), train_dataset.classes


def create_data_loaders(dataset_root: Path):
    train_subset, val_subset, class_names = _build_subsets(dataset_root)
    loader_kwargs = {
        'batch_size': BATCH_SIZE,
        'num_workers': 2,
        'pin_memory': torch.cuda.is_available(),
    }
    train_loader = DataLoader(train_subset, shuffle=True, **loader_kwargs)
    val_loader = DataLoader(val_subset, shuffle=False, **loader_kwargs)
    return train_loader, val_loader, class_names


def save_metadata(class_names):
    metadata = {
        "class_names": class_names,
        "image_size": IMAGE_SIZE,
        "model_path": str(MODEL_PATH.relative_to(Path.cwd())),
        "created_at": time.strftime("%Y-%m-%d %H:%M:%S"),
    }
    METADATA_PATH.write_text(json.dumps(metadata, indent=2))
    return metadata


In [3]:
def train_one_epoch(model: nn.Module, data_loader: DataLoader, optimizer, criterion):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for inputs, labels in data_loader:
        inputs = inputs.to(DEVICE)
        labels = labels.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
        predictions = outputs.argmax(dim=1)
        correct += (predictions == labels).sum().item()
        total += labels.size(0)
    return running_loss / total, correct / total


def evaluate(model: nn.Module, data_loader: DataLoader, criterion):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs = inputs.to(DEVICE)
            labels = labels.to(DEVICE)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * inputs.size(0)
            predictions = outputs.argmax(dim=1)
            correct += (predictions == labels).sum().item()
            total += labels.size(0)
    return running_loss / total, correct / total


def fit(model: nn.Module, train_loader: DataLoader, val_loader: DataLoader):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
    best_val_accuracy = 0.0
    best_state = None
    history = []
    for epoch in range(1, EPOCHS + 1):
        train_loss, train_acc = train_one_epoch(model, train_loader, optimizer, criterion)
        val_loss, val_acc = evaluate(model, val_loader, criterion)
        history.append({
            "epoch": epoch,
            "train_loss": train_loss,
            "train_acc": train_acc,
            "val_loss": val_loss,
            "val_acc": val_acc,
        })
        print(f"Epoch {epoch}/{EPOCHS} - train_loss: {train_loss:.4f} train_acc: {train_acc:.3f} val_loss: {val_loss:.4f} val_acc: {val_acc:.3f}")
        if val_acc > best_val_accuracy:
            best_val_accuracy = val_acc
            best_state = model.state_dict()
    if best_state is not None:
        model.load_state_dict(best_state)
    return history


In [4]:
train_loader, val_loader, class_names = create_data_loaders(DATASET_DIR)
model = create_model(num_classes=len(class_names)).to(DEVICE)
training_history = fit(model, train_loader, val_loader)
torch.save(model.state_dict(), MODEL_PATH)
metadata = save_metadata(class_names)
print(f"Model saved to {MODEL_PATH}")
print(f"Metadata saved to {METADATA_PATH}")


Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to C:\Users\17619/.cache\torch\hub\checkpoints\vgg16-397923af.pth
100%|██████████| 528M/528M [00:12<00:00, 43.5MB/s] 


Epoch 1/12 - train_loss: 1.6584 train_acc: 0.343 val_loss: 1.2940 val_acc: 0.413
Epoch 2/12 - train_loss: 1.5877 train_acc: 0.338 val_loss: 1.3300 val_acc: 0.399
Epoch 3/12 - train_loss: 1.4385 train_acc: 0.365 val_loss: 1.2969 val_acc: 0.430
Epoch 4/12 - train_loss: 1.4332 train_acc: 0.368 val_loss: 1.3057 val_acc: 0.404
Epoch 5/12 - train_loss: 1.3914 train_acc: 0.377 val_loss: 1.2783 val_acc: 0.467
Epoch 6/12 - train_loss: 1.3820 train_acc: 0.384 val_loss: 1.2554 val_acc: 0.453
Epoch 7/12 - train_loss: 1.3722 train_acc: 0.390 val_loss: 1.2738 val_acc: 0.445
Epoch 8/12 - train_loss: 1.3690 train_acc: 0.396 val_loss: 1.2730 val_acc: 0.408
Epoch 9/12 - train_loss: 1.3594 train_acc: 0.390 val_loss: 1.2509 val_acc: 0.448
Epoch 10/12 - train_loss: 1.3523 train_acc: 0.395 val_loss: 1.2749 val_acc: 0.421
Epoch 11/12 - train_loss: 1.3477 train_acc: 0.396 val_loss: 1.2525 val_acc: 0.445
Epoch 12/12 - train_loss: 1.3442 train_acc: 0.388 val_loss: 1.2462 val_acc: 0.451
Model saved to h:\DogEmot

In [5]:
reloaded_model = create_model(num_classes=len(class_names)).to(DEVICE)
reloaded_model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))
reloaded_model.eval()
sample_inputs, _ = next(iter(val_loader))
with torch.no_grad():
    probabilities = torch.softmax(reloaded_model(sample_inputs.to(DEVICE)), dim=1)
    scores, indices = torch.max(probabilities, dim=1)
for idx in range(min(5, sample_inputs.size(0))):
    label = class_names[indices[idx].item()]
    confidence = scores[idx].item()
    print(f"Prediction {idx + 1}: {label} ({confidence:.2%})")


C:\Users\17619\AppData\Local\Temp\ipykernel_28704\2239903890.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  reloaded_model.load_state_dict(torch.load(MODEL_PATH, map_lo

Prediction 1: sad (33.53%)
Prediction 2: happy (31.06%)
Prediction 3: sad (36.31%)
Prediction 4: happy (28.55%)
Prediction 5: happy (33.47%)
